# H_22082024
# Analisis del volumen profile para encontrar zonas de alta probabilidad y como se comporta el precio en esas zonas.

In [24]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
import json
import plotly.express as px
import plotly.graph_objects as go

In [25]:
# Variables

STD_MULTIPLIER = 1
ticker = "ES_U"
start_dt = "2021-05-25"
end_dt = "2024-06-28"
minute_data = "ES_U_M1_200912160000_202408012359.csv"
daily_data = "ES_U_Daily_200912160000_202408010000.csv"

## Recopilación de datos, manipulación y limpieza

In [26]:
def data_from_mt5():
    # connect to MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()

    # set time zone to UTC
    timezone = pytz.timezone("Etc/UTC")
    # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
    utc_from = datetime.strptime(start_dt, "%Y-%m-%d")
    utc_to = datetime.strptime(end_dt, "%Y-%m-%d")#datetime(2024, 8, 1, tzinfo=timezone)
    # request AUDUSD ticks within 11.01.2020 - 11.01.2020
    ohlcv = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_M1, utc_from, utc_to)
    daily_ohlcv = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_D1, utc_from, utc_to)
    print(ohlcv)

    mt5.shutdown()

    df = pd.DataFrame(ohlcv)
    df['time']=pd.to_datetime(df['time'], unit='s')

    # adaptamos el dataframe D1 para luego hacer los analisis de las sesiones
    daily_df = pd.DataFrame(daily_ohlcv)
    daily_df['time']=pd.to_datetime(daily_df['time'], unit='s')

    df = df.set_index('time')
    del df['real_volume']
    del df['spread']

    daily_df = daily_df.set_index('time')
    del daily_df['real_volume']
    del daily_df['spread']

    return df, daily_df

def data_from_csv():
    df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/'+minute_data, sep='\t')
    df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    del df['<TIME>']
    del df['<VOL>']
    del df['<SPREAD>']
    df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    df = df.set_index('time')

    daily_df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/'+daily_data, sep='\t')
    #df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    #del df['<TIME>']
    del daily_df['<VOL>']
    del daily_df['<SPREAD>']
    daily_df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    daily_df = daily_df.set_index('time')
    #daily_df = df.resample('1B').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'tick_volume': 'sum'})

    return df, daily_df

df, daily_df = data_from_csv()

df

,open,high,low,close,tick_volume
time,,,,,
2009-12-16 00:00:00,1022.75,1023.00,1022.75,1022.75,1218
2009-12-16 00:01:00,1022.75,1023.00,1022.75,1022.75,776
2009-12-16 00:02:00,1023.00,1023.00,1022.75,1023.00,76
2009-12-16 00:03:00,1022.75,1023.00,1022.75,1022.75,256
2009-12-16 00:04:00,1023.00,1023.00,1022.75,1023.00,97
...,...,...,...,...,...
2024-08-01 23:55:00,5468.00,5469.00,5466.00,5468.50,30
2024-08-01 23:56:00,5468.50,5469.75,5468.00,5469.75,43
2024-08-01 23:57:00,5469.75,5469.75,5469.50,5469.75,11


In [27]:
def create_market_profile(data, getPOC=True):
    profile = data.groupby('close')['tick_volume'].sum().reset_index()
    total_volume = profile['tick_volume'].sum()
    profile['volume_cumsum'] = profile['tick_volume'].cumsum()

    value_area_cutoff = total_volume * 0.70
    value_area_df = profile[profile['volume_cumsum'] <= value_area_cutoff]
    POC = 0
    if getPOC:
        POC = profile.loc[profile['tick_volume'].idxmax(), 'close']
    else:
        POC = profile.loc[profile['tick_volume'].idxmin(), 'close']

    return profile, value_area_df, POC

def plot_market_profile(profile, value_area_df, POC):
    plt.figure(figsize=(10, 6))
    plt.barh(profile['close'], profile['tick_volume'], color='blue', edgecolor='black')
    plt.barh(value_area_df['close'], value_area_df['tick_volume'], color='green', edgecolor='black')
    plt.axhline(POC, color='red', linestyle='--', label=f'POC: {POC}')

    plt.xlabel('Volume')
    plt.ylabel('Price')
    plt.title(f'Market Profile')
    plt.legend()
    plt.show()

In [28]:
def get_no_fair_range_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] < threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

def get_max_vol_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] > threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

In [29]:
# iteramos cada dia, y cada zona
def get_no_fair_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, MIN = create_market_profile(day, False)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        no_fair_value_zone = get_no_fair_range_zone(profile, threshold)

        if no_fair_value_zone == False:
            output = {
                "time": index1,
                "min_zone_high": np.nan,
                "min_zone_low": np.nan,
                "MIN": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "min_zone_high": no_fair_value_zone[1],
                "min_zone_low": no_fair_value_zone[0],
                "MIN": MIN,
            }
            out.append(output)

    return out

# iteramos cada dia, y cada zona
def get_max_vol_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        max_value_zone = get_max_vol_zone(profile, threshold)

        if max_value_zone == False:
            output = {
                "time": index1,
                "max_zone_high": np.nan,
                "max_zone_low": np.nan,
                "POC": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "max_zone_high": max_value_zone[1],
                "max_zone_low": max_value_zone[0],
                "POC": POC,
            }
            out.append(output)

    return out

## Analisis de datos

In [30]:
#zones = get_no_fair_zone_by_day(df)
max_zones = get_max_vol_zone_by_day(df)
df_max_zones = pd.DataFrame(max_zones)
#df_zones['time'].astype('datetime64[ns]')
df_max_zones = df_max_zones.set_index('time')
df_max_zones.index.astype('datetime64[ns]')

#zones = get_no_fair_zone_by_day(df)
min_zones = get_no_fair_zone_by_day(df)
df_min_zones = pd.DataFrame(min_zones)
#df_zones['time'].astype('datetime64[ns]')
df_min_zones = df_min_zones.set_index('time')
df_min_zones.index.astype('datetime64[ns]')

main_df_2 = pd.merge(df_max_zones, df_min_zones, on=daily_df.index)
main_df_2 = main_df_2.set_index('key_0')
main_df = pd.merge(daily_df, main_df_2, on=daily_df.index)
#daily_df.join(df_zones, on=daily_df.index , how='inner')
main_df = main_df.set_index('key_0')
main_df.dropna()
main_df

#main_df.to_csv( TICKER+'_H22082024_Output_Data.csv', index=True)

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN
key_0,,,,,,,,,,,
2009.12.16,1022.75,1027.25,1016.25,1025.50,1305332,1025.50,1025.50,1025.50,1024.25,1021.00,1027.00
2009.12.17,1025.50,1026.50,1010.00,1011.00,1702224,1012.75,1012.50,1020.75,1020.50,1014.00,1016.00
2009.12.18,1011.00,1018.50,1005.50,1007.50,1766459,1011.00,1010.50,1010.00,1018.25,1013.25,1017.75
2009.12.19,1007.50,1015.00,1006.75,1014.00,776860,1013.75,1013.75,1013.75,1009.50,1007.00,1014.00
2009.12.21,1014.75,1027.25,1012.75,1026.25,675895,1026.50,1025.50,1026.50,1023.00,1012.75,1013.25
...,...,...,...,...,...,...,...,...,...,...,...
2024.07.26,5447.25,5528.00,5445.50,5499.00,60124,5481.50,5480.25,5480.25,5477.00,5445.75,5453.00
2024.07.29,5498.50,5534.25,5481.25,5504.50,50354,5509.00,5508.00,5507.50,5500.50,5483.75,5529.75
2024.07.30,5505.50,5527.50,5433.00,5452.50,55695,5512.00,5511.50,5474.25,5488.50,5474.75,5495.00


### SETUP_1. Cierre de rango en la siguiente sesión

Analizaremos si en la siguiente sesión, el precio cierra el rango del dia anterior.

In [31]:
def num1_next_session_close_zone(df):
    results = []

    for i in range(len(df.index)-1):
        if i == 0: pass
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        # Comprobar si se cumple la condición
        if zone_high <= high and zone_high >= low and zone_low >= low and zone_low <= high:
            results.append(True)  # Cumple la condición
        else:
            results.append(False)  # No cumple la condición

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_1'] = results

    return df['SETUP_1'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de cierre de la zona: ", num1_next_session_close_zone(main_df))

Porcentaje de cierre de la zona:  SETUP_1
True       68.2777399591559%
False    31.722260040844112%
Name: proportion, dtype: object


### SETUP_2. Cierre de medio rango en la siguiente sesion

Analizaremos si en la siguiente sesión, el precio cierra la midat del rango.

In [32]:
def num2_next_session_close_half_zone(df):
    results = []

    for i in range(len(df.index) - 1):
        if i == 0: pass
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']
        zone_mid = ((zone_high - zone_low) / 2) + zone_low

        if apertura >= zone_high:
            # Mitad superior
            if zone_high <= high and zone_high >= low and zone_mid >= low and zone_mid <= high:
                results.append(True)
            else:
                results.append(False)
        elif apertura <= zone_low:
            # Mitad inferior
            if zone_mid <= high and zone_mid >= low and zone_low >= low and zone_low <= high:
                results.append(True)
            else:
                results.append(False)
        else:
            results.append(False)
    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_2'] = results

    return df['SETUP_2'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de cierre de media zona: ", num2_next_session_close_half_zone(main_df))

Porcentaje de cierre de media zona:  SETUP_2
True     65.35057862491492%
False    34.64942137508509%
Name: proportion, dtype: object


### SETUP_3. Desde apertura a extremo de zona

Analisis de la cantidad de veces que el precio recorre el rango entre la apertura de la sesion y un valor extremo.

Depende de si la apertura es por debajo o per encima de la zona de alta capitalización, buscaremos largos o cortos.

In [33]:
def num3_from_open_to_range_max_volume(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        if apertura >= zone_high:
            # Bajista
            if zone_low <= high and zone_low >= low:
                results.append(True)
            else:
                results.append(False)
        elif apertura <= zone_low:
            # Alcista
            if zone_high <= high and zone_high >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_3'] = results

    return df['SETUP_3'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de Apertura-Extremo: ", num3_from_open_to_range_max_volume(main_df))

Porcentaje de Apertura-Extremo:  SETUP_3
True      64.23871114136601%
False    35.761288858633996%
Name: proportion, dtype: object


### SETUP_4. Recorrido a POC anterior session desde apertura 

Analisis de la cantidad de veces que se testea el valor POC de la anterior sesion.

In [34]:
def num4_POC_test(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['POC']

        if apertura >= poc:
            # Bajista
            if poc <= high and poc >= low:
                results.append(True)
            else:
                results.append(False)

        elif apertura <= poc:
            # Alcista
            if poc <= high and poc >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_4'] = results

    return df['SETUP_4'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de test POC: ", num4_POC_test(main_df))

Porcentaje de test POC:  SETUP_4
True      69.8888132516451%
False    30.11118674835489%
Name: proportion, dtype: object


### Nivel de mayor probabilidad estadistica: SETUP_4

A continuación, sacaremos más estadisticas que complementarán el estudio.

In [35]:
def cummulative_not_setup4_true(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == False:
            tmp = tmp + 1
        else:
            results.append(tmp)
            tmp = 0

    nparr = np.array(results)

    return nparr.max()

print("Maximos fallos consecutivos de SETUP 4: ", cummulative_not_setup4_true(main_df))

Maximos fallos consecutivos de SETUP 4:  7


In [36]:
def setup4_range_stddev(df):
    df['SETUP_4_range'] = abs(df['POC'] - df['open'])

    true_df = df.query('SETUP_4 == True')
    tmean = true_df['SETUP_4_range'].mean()
    tstddev = true_df['SETUP_4_range'].std()

    false_df = df.query('SETUP_4 == False')
    fmean = false_df['SETUP_4_range'].mean()
    fstddev = false_df['SETUP_4_range'].std()

    return tmean, fmean, tstddev, fstddev

print("Datos de rango MEAN, STDDEV\n", setup4_range_stddev(main_df))

Datos de rango MEAN, STDDEV
 (14.219237012987014, 10.349697885196374, 21.45621498123276, 15.243227431352127)


In [37]:
main_df

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN,SETUP_1,SETUP_2,SETUP_3,SETUP_4,SETUP_4_range
key_0,,,,,,,,,,,,,,,,
2009.12.16,1022.75,1027.25,1016.25,1025.50,1305332,1025.50,1025.50,1025.50,1024.25,1021.00,1027.00,True,True,True,True,2.75
2009.12.17,1025.50,1026.50,1010.00,1011.00,1702224,1012.75,1012.50,1020.75,1020.50,1014.00,1016.00,True,True,True,False,4.75
2009.12.18,1011.00,1018.50,1005.50,1007.50,1766459,1011.00,1010.50,1010.00,1018.25,1013.25,1017.75,True,True,True,True,1.00
2009.12.19,1007.50,1015.00,1006.75,1014.00,776860,1013.75,1013.75,1013.75,1009.50,1007.00,1014.00,True,True,True,True,6.25
2009.12.21,1014.75,1027.25,1012.75,1026.25,675895,1026.50,1025.50,1026.50,1023.00,1012.75,1013.25,True,True,True,True,11.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.07.26,5447.25,5528.00,5445.50,5499.00,60124,5481.50,5480.25,5480.25,5477.00,5445.75,5453.00,False,False,False,False,33.00
2024.07.29,5498.50,5534.25,5481.25,5504.50,50354,5509.00,5508.00,5507.50,5500.50,5483.75,5529.75,True,True,True,True,9.00
2024.07.30,5505.50,5527.50,5433.00,5452.50,55695,5512.00,5511.50,5474.25,5488.50,5474.75,5495.00,True,True,True,True,31.25


In [38]:
def low_prob_MIN(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['MIN']

        if high >= poc >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YL(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yl = df.iloc[i]['low']

        if high >= yl >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YH(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yh = df.iloc[i]['high']

        if high >= yh >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YO(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yo = df.iloc[i]['open']

        if high >= yo >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100


print("Porcentaje de test MIN Volume: ", low_prob_MIN(main_df))
print("Porcentaje de test YL: ", low_prob_YL(main_df))
print("Porcentaje de test YH: ", low_prob_YH(main_df))
print("Porcentaje de test YO: ", low_prob_YO(main_df))

Porcentaje de test MIN Volume:  49.773036768043575
Porcentaje de test YL:  44.802541988197916
Porcentaje de test YH:  55.44711756695415
Porcentaje de test YO:  50.317748524738995


In [39]:
#main_df.to_csv("OUTPUT.csv")

## Relación entre el Point Of Control (POC) y la zona de mayor transito de ticks (PHF).

In [40]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta

In [41]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

main_df = main_df.loc[start_dt:]
main_df['PHF'] = np.NaN # Create new column to allocate data
# iterate main_df for adding zone to data
for index, row in main_df.iterrows():
    dt1 = datetime.strptime(index.replace('.', '-'), "%Y-%m-%d")
    dt2 = dt1 + timedelta(hours=23, minutes=59, seconds=59) 
    
    ticks = mt5.copy_ticks_range(ticker, dt1, dt2, mt5.COPY_TICKS_ALL)

    # create DataFrame out of the obtained data
    df = pd.DataFrame(ticks)
    # convert time in seconds into the datetime format
    df['time']=pd.to_datetime(df['time'], unit='s')

    df = df.set_index('time')
    del df['last']
    del df['flags']
    del df['volume']
    del df['time_msc']
    del df['volume_real']

    flag_full_session = True #Flag for analising full day, not only the opening
    zone_output = 0 # Output from K-Means algorithm
    for index1, day in df.groupby(df.index.date):
        # Iterating each session
        start_session = pd.to_datetime(index1.strftime('%Y-%m-%d') + ' ' + '15:30:00')
        end_session = pd.to_datetime(index1.strftime('%Y-%m-%d') + ' ' + '16:00:00')

        # DF contains the session to analise.
        if flag_full_session:
            refdf = day
        else:
            refdf = day.loc[start_session:end_session]
        
        # create a MinMaxScaler object
        scaler = MinMaxScaler()

        # fit and transform the data
        normalized_data = scaler.fit_transform(refdf)

        # create a new DataFrame with the normalized data
        ndf = pd.DataFrame(normalized_data, columns=refdf.columns)

        ndf['ask'] = ndf['ask'].fillna(ndf['bid'])
        ndf['bid'] = ndf['bid'].fillna(ndf['ask'])

        ndf_to_matrix = ndf.values
        # Preparing data for clustering: Normalize time and price to have similar scales
        X_time = np.linspace(0, 1, len(ndf_to_matrix)).reshape(-1, 1)
        X_price = (ndf['ask'].values - np.min(ndf['ask'])) / (np.max(ndf['ask']) - np.min(ndf['ask']))
        X_cluster = np.column_stack((X_time, X_price))

        # Applying KMeans clustering
        num_clusters = 1
        kmeans = KMeans(n_clusters=num_clusters)
        kmeans.fit(ndf_to_matrix)

        # Extract cluster centers and rescale back to original price range
        cluster_centers = kmeans.cluster_centers_[:, 1] * (np.max(refdf['ask']) - np.min(refdf['ask'])) + np.min(refdf['ask'])
        
        zones = cluster_centers.tolist()
        #output = {
        #    "date": index1.strftime('%Y-%m-%d'),
        #    "zones": zones,
        #}
        zone_output = cluster_centers

    # Add output to main_df
    main_df.loc[index, 'PHF'] = zone_output

mt5.shutdown()
main_df

C:\Users\iamfr\AppData\Local\Temp\ipykernel_19424\1204560456.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['PHF'] = np.NaN # Create new column to allocate data
C:\Users\iamfr\AppData\Local\Temp\ipykernel_19424\1204560456.py:55: RuntimeWarning: invalid value encountered in divide
  X_price = (ndf['ask'].values - np.min(ndf['ask'])) / (np.max(ndf['ask']) - np.min(ndf['ask']))
C:\Users\iamfr\AppData\Local\Temp\ipykernel_19424\1204560456.py:55: RuntimeWarning: invalid value encountered in divide
  X_price = (ndf['ask'].values - np.min(ndf['ask'])) / (np.max(ndf['ask']) - np.min(ndf['ask']))
C:\Users\iamfr\AppData\Local\Temp\ipykernel_19424\1204560456.py:55: RuntimeWarning: invalid value encountered in divide
  X_price = (ndf['ask'].valu

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN,SETUP_1,SETUP_2,SETUP_3,SETUP_4,SETUP_4_range,PHF
key_0,,,,,,,,,,,,,,,,,
2021.01.01,3948.50,3984.50,3948.00,3980.25,598078,3983.00,3983.00,3975.50,3973.00,3948.75,3948.75,True,True,True,True,27.00,0.000000
2021.01.04,3980.25,4006.25,3897.75,3900.75,1039746,3913.75,3912.25,3923.50,4005.75,3989.50,3985.50,True,True,True,True,56.75,0.000000
2021.01.05,3900.75,3943.25,3877.75,3930.00,1713166,3924.50,3923.50,3920.25,3942.25,3925.50,3907.00,True,True,True,True,19.50,0.000000
2021.01.06,3930.00,3998.50,3913.00,3990.75,1607574,3957.50,3957.00,3947.00,3925.75,3915.50,3922.75,False,False,False,False,17.00,0.000000
2021.01.07,3991.00,4038.00,3960.25,4026.50,1808723,4029.25,4028.75,3971.25,4003.50,3994.50,3982.00,True,True,True,False,19.75,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.07.26,5447.25,5528.00,5445.50,5499.00,60124,5481.50,5480.25,5480.25,5477.00,5445.75,5453.00,False,False,False,False,33.00,5486.483027
2024.07.29,5498.50,5534.25,5481.25,5504.50,50354,5509.00,5508.00,5507.50,5500.50,5483.75,5529.75,True,True,True,True,9.00,5511.779222
2024.07.30,5505.50,5527.50,5433.00,5452.50,55695,5512.00,5511.50,5474.25,5488.50,5474.75,5495.00,True,True,True,True,31.25,5486.863414


In [42]:
main_df = main_df.drop(main_df[main_df.PHF <= 0.00].index)
main_df

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN,SETUP_1,SETUP_2,SETUP_3,SETUP_4,SETUP_4_range,PHF
key_0,,,,,,,,,,,,,,,,,
2021.05.24,4418.50,4470.00,4408.75,4465.75,615680,4466.25,4464.25,4464.25,4442.75,4411.00,4421.00,True,False,False,True,45.75,4163.540870
2021.05.25,4466.00,4483.50,4451.25,4463.75,1147871,4468.75,4468.25,4464.00,4482.75,4477.75,4482.75,True,True,True,True,2.00,4189.433290
2021.05.26,4463.75,4474.50,4447.75,4466.50,1058371,4460.25,4459.00,4460.25,4474.25,4463.25,4473.75,True,True,True,True,3.50,4184.951946
2021.05.27,4466.50,4480.75,4446.25,4473.50,1086011,4463.25,4462.25,4459.75,4459.25,4448.25,4448.75,False,False,False,False,6.75,4185.721504
2021.05.28,4473.50,4488.50,4463.75,4482.50,1063084,4484.75,4484.75,4464.75,4476.75,4469.75,4488.25,False,False,False,False,8.75,4201.133451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.07.26,5447.25,5528.00,5445.50,5499.00,60124,5481.50,5480.25,5480.25,5477.00,5445.75,5453.00,False,False,False,False,33.00,5486.483027
2024.07.29,5498.50,5534.25,5481.25,5504.50,50354,5509.00,5508.00,5507.50,5500.50,5483.75,5529.75,True,True,True,True,9.00,5511.779222
2024.07.30,5505.50,5527.50,5433.00,5452.50,55695,5512.00,5511.50,5474.25,5488.50,5474.75,5495.00,True,True,True,True,31.25,5486.863414


In [43]:
def num5_PHF_test(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        phf = df.iloc[i]['PHF']

        if apertura >= phf:
            # Bajista
            if phf <= high and phf >= low:
                results.append(True)
            else:
                results.append(False)

        elif apertura <= phf:
            # Alcista
            if phf <= high and phf >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_5'] = results

    return df['SETUP_5'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de test POC: ", num5_PHF_test(main_df))

Porcentaje de test POC:  SETUP_5
False     87.91500664010624%
True     12.084993359893758%
Name: proportion, dtype: object


In [44]:
def cummulative_not_setup5_true(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == False:
            tmp = tmp + 1
        else:
            results.append(tmp)
            tmp = 0

    nparr = np.array(results)

    return nparr.max()

print("Maximos fallos consecutivos de SETUP 5: ", cummulative_not_setup5_true(main_df))

Maximos fallos consecutivos de SETUP 5:  4


In [45]:
#Calculate crossed probability of SETUP 4 || 5
def cross_prob_setup4_OR_5(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == True or row['SETUP_5'] == True:
            tmp = tmp + 1
    
    return (tmp/len(df))*100

#Calculate crossed probability of SETUP 4 && 5
def cross_prob_setup4_AND_5(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == True and row['SETUP_5'] == True:
            tmp = tmp + 1
    
    return (tmp/len(df))*100

print("Probabilidad de SETUP 4 or SETUP 5: ", cross_prob_setup4_OR_5(main_df))
print("Probabilidad de SETUP 4 and SETUP 5: ", cross_prob_setup4_AND_5(main_df))

Probabilidad de SETUP 4 or SETUP 5:  81.80610889774236
Probabilidad de SETUP 4 and SETUP 5:  10.756972111553784
